In [8]:
from dask import dataframe as dd
from datetime import datetime

In [9]:
def parse_datetime(dt):
    date_format = '%Y-%m-%d %H:%M:%S.%f'
    try:
        dt = datetime.strptime(dt[:-4], date_format).timestamp()
    except ValueError:
        dt = datetime.strptime(dt[:-4], date_format[:-3]).timestamp()

    dt *= 1000.0
    dt = int(dt)

    dt -= 1648806250315

    return dt

216581262


In [ ]:
def parse_timestamp(timestamp):
    date_format = '%Y-%m-%d %H'

    timestamp /= 1000
    timestamp = int(timestamp)
    timestamp += 1648806250

    dt = datetime.fromtimestamp(timestamp).strftime(date_format)
    dt = datetime.strptime(dt, date_format)

    return dt

In [3]:
def parse_pixel_color(pixel_color):
    hex_to_key = {
        "#000000": 0,
        "#00756F": 1,
        "#009EAA": 2,
        "#00A368": 3,
        "#00CC78": 4,
        "#00CCC0": 5,
        "#2450A4": 6,
        "#3690EA": 7,
        "#493AC1": 8,
        "#515252": 9,
        "#51E9F4": 10,
        "#6A5CFF": 11,
        "#6D001A": 12,
        "#6D482F": 13,
        "#7EED56": 14,
        "#811E9F": 15,
        "#898D90": 16,
        "#94B3FF": 17,
        "#9C6926": 18,
        "#B44AC0": 19,
        "#BE0039": 20,
        "#D4D7D9": 21,
        "#DE107F": 22,
        "#E4ABFF": 23,
        "#FF3881": 24,
        "#FF4500": 25,
        "#FF99AA": 26,
        "#FFA800": 27,
        "#FFB470": 28,
        "#FFD635": 29,
        "#FFF8B8": 30,
        "#FFFFFF": 31,
    }

    return hex_to_key[pixel_color]

## Оптимизиране на оригиналната информация. Премахване на колоната user_id

In [4]:
ddf = dd.read_csv('..\\data\\raw\\2022_place_canvas_history.csv', converters={"timestamp": parse_datetime, "pixel_color": parse_pixel_color})
ddf_optimized = ddf.drop('user_id', axis=1).compute()
ddf_optimized.astype({'timestamp': 'uint32'}).dtypes
ddf_optimized.astype({'pixel_color': 'uint8'}).dtypes
ddf_optimized = ddf_optimized.sort_values('timestamp')

ddf_optimized.head()

,timestamp,pixel_color,coordinate
0,216581262,5,"826,1048"
1,216583443,17,"583,1031"
2,216584370,11,"1873,558"
3,216647226,2,"1627,255"
4,216665992,17,"49,1478"
5,216670325,23,"408,1863"
6,216684583,17,"111,1582"
7,216690060,11,"1334,1840"
8,216706853,11,"1908,1854"
9,216717115,2,"1504,1722"


In [14]:
ddf_optimized.to_csv('..\\data\\raw\\2022_place_canvas_history_optimized.csv', index=False, single_file=True)

['f:\\Дипломна работа\\code\\data\\raw\\2022_place_canvas_history_optimized.csv']

## Експортиране на колоната user_id в отделен файл

In [ ]:
ddf = dd.read_csv('..\\data\\raw\\2022_place_canvas_history.csv')
ddf_users = ddf.drop(['timestamp','pixel_color','coordinate'],axis=1)
ddf_users.head()

In [ ]:
ddf_users.to_csv('..\\data\\raw\\2022_place_canvas_history_users.csv', index=False, single_file=True)

## Преобразуване на timestamp към datetime и експортиране на datetime колоната в отделен файл

In [10]:
ddf_date = dd.read_csv('..\\data\\raw\\2022_place_canvas_history_optimized.csv')
ddf_date = ddf_date.drop(['pixel_color', 'coordinate'], axis=1)
ddf_date.head()

,timestamp
0,0
1,12356
2,16311
3,21388
4,34094


In [13]:
ddf_date = ddf_date.rename(columns={'timestamp': 'datetime'})
ddf_date['datetime'] = ddf_date['datetime'].map(parse_timestamp)
ddf_date.head()

,datetime
0,2022-04-01 12:00:00
1,2022-04-01 12:00:00
2,2022-04-01 12:00:00
3,2022-04-01 12:00:00
4,2022-04-01 12:00:00


In [14]:
ddf_date.to_csv('..\\data\\raw\\2022_place_canvas_history_datetime.csv', index=False, single_file=True)

['f:\\Дипломна работа\\code\\data\\raw\\2022_place_canvas_history_datetime.csv']

## Завъртане на pixel_color данните, оставяйки datetime като индекс и преброяване на pixel_color видовете за всяка дата и час

In [102]:
ddf_pivot = dd.read_csv('..\\data\\raw\\2022_place_canvas_history_optimized.csv')
ddf_pivot = ddf_pivot.drop('coordinate', axis=1)
ddf_pivot = ddf_pivot.rename(columns={'timestamp': 'datetime'})

ddf_pivot['datetime'] = ddf_pivot['datetime'].map(parse_timestamp)
ddf_pivot['index'] = ddf_pivot.index
ddf_pivot.head()

,datetime,pixel_color
0,2022-04-01 12:00:00,14
1,2022-04-01 12:00:00,3
2,2022-04-01 12:00:00,7
3,2022-04-01 12:00:00,21
4,2022-04-01 12:00:00,7


In [103]:
ddf_pivot['pixel_color'] = ddf_pivot['pixel_color'].astype('category')
ddf_pivot['pixel_color'] = ddf_pivot.pixel_color.cat.as_known()
ddf_pivot.dtypes

datetime       datetime64[ns]
pixel_color          category
dtype: object

In [169]:
ddf_pivoted = ddf_pivot.pivot_table(index='datetime', columns='pixel_color', values='pixel_color', aggfunc='count')
ddf_pivoted.head()

pixel_color,0,3,6,7,10,14,15,16,18,19,...,20,24,5,9,12,17,22,23,28,30
datetime,,,,,,,,,,,,,,,,,,,,,
2022-04-01 12:00:00,0.0,2.0,3.0,2.0,1.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-04-01 13:00:00,117511.0,21775.0,83172.0,15056.0,24328.0,21971.0,19421.0,7081.0,6169.0,5815.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-04-01 14:00:00,198737.0,33085.0,115221.0,20169.0,38124.0,30806.0,26811.0,12597.0,9609.0,7658.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-04-01 15:00:00,242980.0,35452.0,98686.0,25440.0,42816.0,33479.0,27869.0,17086.0,12235.0,10745.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-04-01 16:00:00,246588.0,40658.0,99270.0,29857.0,49147.0,37166.0,32077.0,20787.0,11657.0,9305.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [174]:
ddf_pivoted.to_csv('..\\data\\raw\\2022_place_canvas_history_pivoted.csv', single_file=True)